# How to connect to OpenAI service via OpenAI Python SDK

First import the OpenAI Python SDK and load your API key from the environment.


In [25]:
from openai import OpenAI
import os
from dotenv import load_dotenv, dotenv_values
from pprint import pprint

config = dotenv_values(".env")

openai_key = config["OPENAI_API_KEY"]
openai_org_id = config["OPENAI_ORG_ID"]
print(
    f"OpenAI api key={os.getenv('OPENAI_API_KEY')[0:3]}...{os.getenv('OPENAI_API_KEY')[-3:]}"
)

OpenAI api key=sk-...Uzy


Lets see what gpt models are available for the API key we loaded.

In [26]:
client_org = OpenAI(
    organization=openai_org_id,
)
models = client_org.models.list().data
models = [m for m in models if "gpt" in m.id.lower()]

# sort models by model id   
models = sorted(models, key=lambda x: x.id)

# print models which contains gpt in the name
pprint(models)



[Model(id='gpt-3.5-turbo', created=1677610602, object='model', owned_by='openai'),
 Model(id='gpt-3.5-turbo-0301', created=1677649963, object='model', owned_by='openai'),
 Model(id='gpt-3.5-turbo-0613', created=1686587434, object='model', owned_by='openai'),
 Model(id='gpt-3.5-turbo-1106', created=1698959748, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-16k', created=1683758102, object='model', owned_by='openai-internal'),
 Model(id='gpt-3.5-turbo-16k-0613', created=1685474247, object='model', owned_by='openai'),
 Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-instruct-0914', created=1694122472, object='model', owned_by='system'),
 Model(id='gpt-4', created=1687882411, object='model', owned_by='openai'),
 Model(id='gpt-4-0314', created=1687882410, object='model', owned_by='openai'),
 Model(id='gpt-4-0613', created=1686588896, object='model', owned_by='openai'),
 Model(id='gpt-4-1106-preview', created=1

Test the direct connection to OpenAI service via OpenAI Python SDK.

In [27]:
oai_client = OpenAI(
    api_key=openai_key
)


response = oai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair.",
        },
        {
            "role": "user",
            "content": "Compose a poem that explains the concept of recursion in programming.",
        },
    ],
)

pprint(response.choices[0].message.content)

ChatCompletionMessage(content="In the realm of code, there lies a tale,\nOf a concept called recursion, never to fail.\nWith elegance, it dances and twirls,\nUnveiling the magic of loops in pearls.\n\nIt starts with a single function's might,\nA journey that expands into the night.\nLike a mirror reflecting its own reflection,\nRecursion captures beauty in its inception.\n\nPicture a labyrinth, twisting and turning,\nAn intricate maze for our minds yearning.\nEach step forward, a call to the past,\nA loop within a loop, steadfast.\n\nA function begins its initial quest,\nUnwinding the problem, its heart fully dressed.\nIt takes its input, so humble and small,\nAnd tackles the task, fearsome and tall.\n\nBut midway through, it takes a detour,\nTrusts itself to solve, unlocks a new door.\nIt calls itself once again, not in despair,\nBut with hope and faith in the recursive air.\n\nLike a bird soaring, it flies through the sky,\nUnfolding layers, as it gently glides by.\nEach nested level

## Create a request to the OpenAI via promptsail proxy

Run the docker and go to PromptSail UI http://promptsail.local/


At the application start we created two test projects with OpenAI API mappings. We will use project1 for this example.

* [project1](http://promptsail.local/ui/project/project1) -> https://api.openai.com/v1

You can create your own project if you want.



In [28]:

ps_api_base = "http://promptsail.local:8000/project1/"


ps_client = OpenAI(
    base_url=ps_api_base,
    api_key=openai_key,
    max_retries=0
)


response = ps_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": "Yoda assistant you are, skilled in explaining complex life and phisopical matters.",
        },
        {
            "role": "user",
            "content": "What number 42 means",
        },
    ],
)

pprint(response.choices[0].message.content)



APIConnectionError: Connection error.